In [36]:
from keras.applications.resnet_v2 import ResNet50V2
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

import numpy as np
import tensorflow as tf

In [28]:
# TO-DO: add argparse when converting to script
num_classes = 4
batch_size = 8
epochs = 10
checkpoint = ''
outputPath = 'output/'
runID = 'test1'
runPath = outputPath + runID
print('Output: ' + runPath)

Output: output/test1


In [11]:
# load data
x_train = np.load('data/x_train.npy')
x_test = np.load('data/x_test.npy')
y_train = np.load('data/y_train.npy')
y_test = np.load('data/y_test.npy')

In [25]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [37]:
def get_callbacks(runPath):
    callbacks = []
    lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.7, patience=5, min_lr=0.00009, min_delta=1e-2)
    callbacks.append(lr_schedule) # reduce learning rate when stuck

    # Callback: save checkpoints '/cp-{epoch:02d}-{val_loss:.2f}.ckpt'
    checkpoint_path = runPath + '/cp-{epoch:02d}-{val_loss:.2f}.hdf5'
    callbacks.append(tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
        verbose=1, save_best_only=False, save_weights_only=True, mode='min', period=1))

    class SaveAsCKPT(keras.callbacks.Callback):
        def __init__(self):
            self.saver = tf.train.Saver()
            self.sess = keras.backend.get_session()

        def on_epoch_end(self, epoch, logs=None):
            checkpoint_path = runPath + '/cp-{:02d}.ckpt'.format(epoch)
            save_path = self.saver.save(self.sess, checkpoint_path)
    callbacks.append(SaveAsCKPT())

    return callbacks


In [38]:
# TO-DO: add checkpoint
model = ResNet50V2(include_top=False, weights='imagenet', input_shape=(224, 224, 3), classes=num_classes)
opt = Adam(learning_rate=0.0001, amsgrad=True)
callbacks = get_callbacks(runPath)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy']) # TO-DO: add additional metrics for COVID-19
print('Ready for training!')
datagen = ImageDataGenerator(rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
                             # randomly shift images horizontally (fraction of total width)
                             width_shift_range=0.1,
                             # randomly shift images vertically (fraction of total height)
                             height_shift_range=0.1,
                             brightness_range=[0.8,1.2], # 1.0 means no change, >1 increase in brightness
                             # set mode for filling points outside the input boundaries
                             fill_mode='nearest',
                             cval=0.,  # value used for fill_mode = "constant"
                             horizontal_flip=True,  # randomly flip images
                             vertical_flip=False)  # randomly flip images

datagen.fit(x_train)

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(x_train, y_train,
                                 batch_size=batch_size),
                    callbacks=callbacks,
                    epochs=epochs,
                    validation_data=(x_test, y_test))

Ready for training!


AttributeError: 'Model' object has no attribute '_in_multi_worker_mode'

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])